# BibRec

### RF Features: Country, State

In [14]:
from bibrec.server.Utils import get_books, get_users, get_ratings

books = get_books("data/BX-Books.csv")
users = get_users("data/BX-Users.csv")
ratings = get_ratings("data/BX-Book-Ratings.csv")

/run/media/fab/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/bibrec/server/Utils.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(path, sep=";", encoding="latin-1")
/run/media/fab/Dev/repos/FH-SBG/MMT1/RES/recommender-systems/bibrec/server/Utils.py:24: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  location_seperated = users.location.str.split(',', 2, expand=True)


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [16]:
explicit_ratings = ratings[ratings.book_rating != 0]
explicit_ratings

,user_id,isbn,book_rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276736,3257224281,8
7,276737,0600570967,6
...,...,...,...
1149773,276704,0806917695,5
1149775,276704,1563526298,9
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [17]:
most_rated_books = explicit_ratings.groupby('isbn').user_id.count().sort_values(ascending=False)
df_ratings = most_rated_books[:1000]
df_ratings = df_ratings.reset_index()
# user_id => nr of ratings
df_ratings = df_ratings.drop('user_id',axis=1)
df_ratings

,isbn
0,0316666343
1,0971880107
2,0385504209
3,0312195516
4,0679781587
...,...
995,0679459626
996,0061097861
997,0875421180
998,0446611921


In [18]:
df_ratings = df_ratings.merge(explicit_ratings, on='isbn', how='left')
df_ratings

,isbn,user_id,book_rating
0,0316666343,277439,7
1,0316666343,277752,7
2,0316666343,278633,9
3,0316666343,638,10
4,0316666343,709,3
...,...,...,...
68436,0399148639,235105,10
68437,0399148639,252475,3
68438,0399148639,258553,9
68439,0399148639,267180,10


In [19]:
# merge datasets
df = df_ratings.merge(users[['user_id', 'country', 'state']])
df.head()

,isbn,user_id,book_rating,country,state
0,0316666343,277439,7,usa,oregon
1,0671027360,277439,7,usa,oregon
2,0316666343,277752,7,usa,louisiana
3,0679734775,277752,8,usa,louisiana
4,0316666343,278633,9,usa,utah


In [20]:
country = pd.get_dummies(df['country'],drop_first=True)

In [21]:
state = pd.get_dummies(df['state'],drop_first=True)

In [22]:
df = pd.concat([df, country, state], axis = 1)
df.head()

,isbn,user_id,book_rating,country,state,"\n/a\"", greece""",albania,algeria,antarctica,argentina,...,zuid-holland,zuidholland,zurich,zürich,álava,maryland,new york,or,tn,town of bali
0,0316666343,277439,7,usa,oregon,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0671027360,277439,7,usa,oregon,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0316666343,277752,7,usa,louisiana,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0679734775,277752,8,usa,louisiana,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0316666343,278633,9,usa,utah,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df = df.loc[:,~df.columns.duplicated()]

In [24]:
test2 = df.drop(['isbn', 'user_id', 'country', 'state'], axis = 1)
test2

,book_rating,"\n/a\"", greece""",albania,algeria,antarctica,argentina,aroostook,aruba,australia,austria,...,zuid-holland,zuidholland,zurich,zürich,álava,maryland,new york,or,tn,town of bali
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68436,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68437,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68438,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
68439,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
X = test2.drop('book_rating', axis = 1) #Features
Y = test2['book_rating']   # Target Variables
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=7)

In [26]:
rfc = RandomForestClassifier(n_estimators=100, min_weight_fraction_leaf= 0)
rfc.fit(X_train, y_train)

RandomForestClassifier(min_weight_fraction_leaf=0)

In [27]:
rfc_pred = rfc.predict(X_test)
print(classification_report(y_test,rfc_pred))
print(accuracy_score(y_test,rfc_pred)*100)

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        70
           2       0.00      0.00      0.00       138
           3       0.00      0.00      0.00       266
           4       0.50      0.00      0.01       378
           5       0.22      0.01      0.02      1750
           6       0.07      0.00      0.01      1532
           7       0.16      0.01      0.02      3393
           8       0.26      0.82      0.40      5258
           9       0.20      0.04      0.06      3835
          10       0.26      0.20      0.23      3913

    accuracy                           0.26     20533
   macro avg       0.17      0.11      0.07     20533
weighted avg       0.21      0.26      0.16     20533

25.76340525008523


/home/fab/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fab/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fab/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
